**классификатор LSTM на Keras+TensorFlow**

Евгений Борисов <esborisov@sevsu.ru>

In [1]:
import numpy as np
from numpy import random as rng

In [2]:
n_samples = 100 # количество примеров
seq_len = 3 # длина последовательности
code_len = 8 # размер элемента последовательности

# генерируем случайные примеры
x_train = rng.rand(n_samples,seq_len,code_len)
x_train.shape

(100, 3, 8)

In [3]:
x_train[:3,:]

array([[[0.76353472, 0.55012196, 0.82111417, 0.36418439, 0.68774139,
         0.3805992 , 0.03053478, 0.87582769],
        [0.26097276, 0.05634855, 0.38736708, 0.71995518, 0.31019273,
         0.66751831, 0.77507078, 0.16826355],
        [0.6230563 , 0.07818055, 0.48198754, 0.666553  , 0.24120078,
         0.80848147, 0.78964496, 0.44356301]],

       [[0.883708  , 0.89068942, 0.15444124, 0.32461685, 0.89290443,
         0.87596616, 0.04899642, 0.05156713],
        [0.07891095, 0.37198181, 0.58410178, 0.11446678, 0.85876973,
         0.50962937, 0.20057392, 0.29164896],
        [0.44981751, 0.41138282, 0.23381301, 0.91775937, 0.13863341,
         0.29556017, 0.40566115, 0.12833582]],

       [[0.35763587, 0.9887792 , 0.25907943, 0.46364602, 0.32771433,
         0.13613475, 0.48436917, 0.04478934],
        [0.30325596, 0.88733471, 0.81369988, 0.70979142, 0.08446903,
         0.13406849, 0.00267232, 0.578026  ],
        [0.72704803, 0.92855294, 0.05107512, 0.70435193, 0.31382023,
       

In [4]:
n_classes = 5

# размечаем примеры случайным образом
target = rng.randint(0,n_classes,n_samples)
y_train = np.eye(n_classes)[target]

y_train.shape

(100, 5)

In [5]:
y_train[:3,:]

array([[0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])

---

In [6]:
import tensorflow as tf
from tabulate import tabulate
from tensorflow.test import is_built_with_cuda
from tensorflow.python.client.device_lib import list_local_devices

def tf_info(tablefmt='html'):
    res = [
        ['tensorflow','%s'%(tf.__version__)],
        ['keras','%s'%(tf.keras.__version__)]
    ]

    if is_built_with_cuda():
        res.extend( [ 
              [ s.strip().split(': ') for s in d.physical_device_desc.split(',')  ]
               for d in list_local_devices() 
              if d.device_type == 'GPU' # and d.name==tf.test.gpu_device_name()
            ][0] )
            
    else:
        res.append(['','no CUDA support'])

    return tabulate( res, tablefmt=tablefmt )

tf_info()

tensorflow,2.4.1
keras,2.4.0
device,0
name,GeForce GTX 1060 6GB
pci bus id,0000:01:00.0
compute capability,6.1


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [8]:
seq_len,code_len

(3, 8)

------

In [ ]:
model = Sequential()
# model.add(Input(shape=(seq_len,code_len)))
model.add(LSTM(32, input_shape=(seq_len,code_len))) 
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
LSTM?

In [ ]:
inputs = tf.random.normal([32, 10, 8])
lstm = LSTM(4)
output = lstm(inputs)
output.shape

In [9]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())

NotImplementedError: Cannot convert a symbolic Tensor (lstm/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

------

In [ ]:
# tf.keras.Input(
#     shape=None,
#     batch_size=None,
#     name=None,
#     dtype=None,
#     sparse=False,
#     tensor=None,
#     ragged=False,
#     **kwargs
# )
#
# tf.keras.layers.LSTM(
#     units,
#     activation="tanh",
#     recurrent_activation="sigmoid",
#     use_bias=True,
#     kernel_initializer="glorot_uniform",
#     recurrent_initializer="orthogonal",
#     bias_initializer="zeros",
#     unit_forget_bias=True,
#     kernel_regularizer=None,
#     recurrent_regularizer=None,
#     bias_regularizer=None,
#     activity_regularizer=None,
#     kernel_constraint=None,
#     recurrent_constraint=None,
#     bias_constraint=None,
#     dropout=0.0,
#     recurrent_dropout=0.0,
#     return_sequences=False,
#     return_state=False,
#     go_backwards=False,
#     stateful=False,
#     time_major=False,
#     unroll=False,
#     **kwargs
# )

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
hist = model.fit(x_train, y_train, batch_size=32, epochs=600, validation_split=.2 )

In [ ]:
from matplotlib import pyplot as plt

k = hist.history.keys()

w,h = 6,4

fig,ax = plt.subplots(1,len(k),figsize=(w*len(k),h))
for i,n in enumerate(k):
    ax[i].plot(hist.history[n],label=n)
    ax[i].grid(True)
    ax[i].legend()
    ax[i].set_ylim([-.1,1.1])